### Columnas de Floor

In [159]:
import pandas as pd
import numpy as np
import re 

In [160]:
data_location = "./properati.csv"
data = pd.read_csv(data_location, sep=",")
print(data.dtypes)
data.head(3)

Unnamed: 0                      int64
operation                      object
property_type                  object
place_name                     object
place_with_parent_names        object
country_name                   object
state_name                     object
geonames_id                   float64
lat-lon                        object
lat                           float64
lon                           float64
price                         float64
currency                       object
price_aprox_local_currency    float64
price_aprox_usd               float64
surface_total_in_m2           float64
surface_covered_in_m2         float64
price_usd_per_m2              float64
price_per_m2                  float64
floor                         float64
rooms                         float64
expenses                      float64
properati_url                  object
description                    object
title                          object
image_thumbnail                object
dtype: objec

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
0,0,sell,PH,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6618237,-58.5088387",-34.661824,...,40.0,1127.272727,1550.000000,NaN,NaN,NaN,http://www.properati.com.ar/15bo8_venta_ph_mat...,"2 AMBIENTES TIPO CASA PLANTA BAJA POR PASILLO,...",2 AMB TIPO CASA SIN EXPENSAS EN PB,https://thumbs4.properati.com/8/BluUYiHJLhgIIK...
1,1,sell,apartment,La Plata,|Argentina|Bs.As. G.B.A. Zona Sur|La Plata|,Argentina,Bs.As. G.B.A. Zona Sur,3432039.0,"-34.9038831,-57.9643295",-34.903883,...,NaN,NaN,NaN,NaN,NaN,NaN,http://www.properati.com.ar/15bob_venta_depart...,Venta de departamento en décimo piso al frente...,VENTA Depto 2 dorm. a estrenar 7 e/ 36 y 37 ...,https://thumbs4.properati.com/7/ikpVBu2ztHA7jv...
2,2,sell,apartment,Mataderos,|Argentina|Capital Federal|Mataderos|,Argentina,Capital Federal,3430787.0,"-34.6522615,-58.5229825",-34.652262,...,55.0,1309.090909,1309.090909,NaN,NaN,NaN,http://www.properati.com.ar/15bod_venta_depart...,2 AMBIENTES 3ER PISO LATERAL LIVING COMEDOR AM...,2 AMB 3ER PISO CON ASCENSOR APTO CREDITO,https://thumbs4.properati.com/5/SXKr34F_IwG3W_...


In [161]:
data.shape

(121220, 26)

In [162]:
#Creación de una nueva columna con la copia de los valores de la columna 'floor'
data['floor_copia'] = data['floor']
data['property_type_copia'] = data['property_type']
#Cambio a minúscula estas dos nuevas columnas creadas
data['description_copia'] = data['description'].astype(str).str.lower()
data['title_copia']= data['title'].astype(str).str.lower()

In [163]:
#Comprobación de valores null en data 'floor_copia'
data['floor_copia'].isnull().value_counts()

True     113321
False      7899
Name: floor_copia, dtype: int64

In [164]:
#Comprobación de valores null en data 'description_copia'
data['description_copia'].isnull().value_counts()

False    121220
Name: description_copia, dtype: int64

In [165]:
#Comprobación de valores null en data 'title_copia'
data['title_copia'].isnull().value_counts()

False    121220
Name: title_copia, dtype: int64

In [166]:
#Categrias en property_type y qué cantidaa hay
data['property_type_copia'].value_counts()

apartment    71065
house        40268
PH            5751
store         4136
Name: property_type_copia, dtype: int64

In [167]:
#agrupo por tipo de propiedad
tipo_propiedad = data.groupby('property_type_copia')

In [168]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 20, 30, 40, 50, 1000, 2000]
pisos = pd.cut(data.floor_copia, bins, right = False)

In [169]:
pisos.value_counts()

[1, 2)          2080
[2, 3)          1542
[3, 4)           914
[9, 15)          743
[4, 5)           658
[5, 6)           484
[6, 7)           483
[7, 8)           355
[8, 9)           248
[50, 1000)       149
[15, 20)         121
[20, 30)          51
[1000, 2000)      34
[30, 40)          20
[40, 50)          13
[0, 1)             0
Name: floor_copia, dtype: int64

In [170]:
data.pivot_table('floor_copia', index = [pisos], columns= 'property_type_copia', aggfunc={'floor_copia': 'count'})

property_type_copia,PH,apartment,house,store
floor_copia,,,,
"[0, 1)",0,0,0,0
"[1, 2)",154,1517,385,24
"[2, 3)",146,1096,290,10
"[3, 4)",63,833,18,0
"[4, 5)",31,625,1,1
"[5, 6)",38,444,2,0
"[6, 7)",42,439,2,0
"[7, 8)",31,320,4,0
"[8, 9)",12,232,4,0


In [171]:
patron = "\spiso:[\s]?(?P<numero>\d[\d]?)"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))

mask_description_notnull = description_series_match.notnull()
data.loc[mask_description_notnull,'floor_copia'] = description_series_match[mask_description_notnull].apply(lambda x: x.group('numero')).astype(float)

In [172]:
data['floor_copia'].isnull().value_counts()

True     112808
False      8412
Name: floor_copia, dtype: int64

In [173]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 20, 30, 40, 50, 1000, 2000]
pisos = pd.cut(data.floor_copia, bins, right = False)
data.pivot_table('floor_copia', index = [pisos], columns= 'property_type_copia', aggfunc={'floor_copia': 'count'})

property_type_copia,PH,apartment,house,store
floor_copia,,,,
"[0, 1)",0,0,0,0
"[1, 2)",158,1538,403,24
"[2, 3)",169,1152,343,12
"[3, 4)",72,865,103,1
"[4, 5)",43,673,35,1
"[5, 6)",38,454,8,0
"[6, 7)",42,448,3,2
"[7, 8)",31,329,4,0
"[8, 9)",12,238,4,2


In [174]:
patron = "piso[\s]?(?P<numero>\d[\d]?)"

busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))

mask_description_notnull = description_series_match.notnull()
data.loc[mask_description_notnull,'floor_copia'] = description_series_match[mask_description_notnull].apply(lambda x: x.group('numero')).astype(float)

In [175]:
data['floor_copia'].isnull().value_counts()

True     109015
False     12205
Name: floor_copia, dtype: int64

In [176]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 20, 30, 40, 50, 1000, 2000]
pisos = pd.cut(data.floor_copia, bins, right = False)
data.pivot_table('floor_copia', index = [pisos], columns= 'property_type_copia', aggfunc={'floor_copia': 'count'})

property_type_copia,PH,apartment,house,store
floor_copia,,,,
"[0, 1)",0,0,0,0
"[1, 2)",161,1825,430,27
"[2, 3)",177,1407,372,17
"[3, 4)",82,1144,136,4
"[4, 5)",46,948,53,5
"[5, 6)",38,560,10,4
"[6, 7)",42,678,6,2
"[7, 8)",31,497,5,1
"[8, 9)",12,425,4,5


In [177]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [178]:
patron = "\s(?P<numero>\d[\d]?)[\s]?[º|°]?\s*piso"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))

mask_description_notnull = description_series_match.notnull()
data.loc[mask_description_notnull,'floor_copia'] = description_series_match[mask_description_notnull].apply(lambda x: x.group('numero')).astype(float)

In [179]:
data.floor_copia.dtypes

dtype('float64')

In [180]:
data['floor_copia'].isnull().value_counts()

True     101137
False     20083
Name: floor_copia, dtype: int64

In [181]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 20, 30, 40, 50, 1000, 2000]
pisos = pd.cut(data.floor_copia, bins, right = False)
data.pivot_table('floor_copia', index = [pisos], columns= 'property_type_copia', aggfunc={'floor_copia': 'count'})

property_type_copia,PH,apartment,house,store
floor_copia,,,,
"[0, 1)",0,0,0,0
"[1, 2)",330,2401,626,88
"[2, 3)",201,1919,461,45
"[3, 4)",96,1683,186,21
"[4, 5)",52,1536,83,18
"[5, 6)",42,1002,27,9
"[6, 7)",45,1181,16,8
"[7, 8)",31,982,12,7
"[8, 9)",13,1190,12,15


In [182]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [183]:
patron = "\s(?P<numero>\d[\d]?)\s*\w{0,5}\s*piso"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))

mask_description_notnull = description_series_match.notnull()
data.loc[mask_description_notnull,'floor_copia'] = description_series_match[mask_description_notnull].apply(lambda x: x.group('numero')).astype(float)

In [184]:
data.floor_copia.dtypes

dtype('float64')

In [185]:
data['floor_copia'].isnull().value_counts()

True     99124
False    22096
Name: floor_copia, dtype: int64

In [186]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 20, 30, 40, 50, 1000, 2000]
pisos = pd.cut(data.floor_copia, bins, right = False)
data.pivot_table('floor_copia', index = [pisos], columns= 'property_type_copia', aggfunc={'floor_copia': 'count'})

property_type_copia,PH,apartment,house,store
floor_copia,,,,
"[0, 1)",0,0,0,0
"[1, 2)",417,2746,774,115
"[2, 3)",230,2219,520,52
"[3, 4)",106,1899,234,23
"[4, 5)",57,1682,121,20
"[5, 6)",45,1114,39,9
"[6, 7)",46,1269,22,10
"[7, 8)",31,1051,16,7
"[8, 9)",13,1259,12,15


In [187]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [188]:
patron = r"(duplex|triplex)"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

numcol_cat_mapper = {'duplex': 2,'triplex': 3}
encontradas1 = encontradas.map(numcol_cat_mapper)


In [189]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [190]:
data['floor_copia'].isnull().value_counts()

True     94666
False    26554
Name: floor_copia, dtype: int64

In [191]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 20, 30, 40, 50, 1000, 2000]
pisos = pd.cut(data.floor_copia, bins, right = False)
data.pivot_table('floor_copia', index = [pisos], columns= 'property_type_copia', aggfunc={'floor_copia': 'count'})

property_type_copia,PH,apartment,house,store
floor_copia,,,,
"[0, 1)",0,0,0,0
"[1, 2)",404,2710,767,115
"[2, 3)",622,4333,2096,70
"[3, 4)",155,2084,513,23
"[4, 5)",57,1672,119,20
"[5, 6)",43,1101,37,9
"[6, 7)",46,1259,22,10
"[7, 8)",30,1040,15,7
"[8, 9)",13,1251,12,15


In [192]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [193]:
patron = r"(primer|segundo|tercer|cuarto|quinto|sexto|s[e|é]ptimo|octavo|noveno|d[e|é]cimo|d[e|é]cimo[\s]?primer|d[e|é]cimo[\s]?segundo|d[e|é]cimo[\s]?tercer|d[e|é]cimo[\s]?cuarto|d[e|é]cimo[\s]?quinto|d[e|é]cimo[\s]?sexto|d[e|é]cimo[\s]?s[e|é]ptimo|d[e|é]cimo[\s]?octavo|d[e|é]cimo[\s]?noveno|vig[e|é]simo)[\s]?piso"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

num_pisos_mapper = {'primer': 1,'segundo':2, 'tercer': 3, 'cuarto': 4, 'quinto': 5, 'sexto': 6, \
                    'septimo':7, 'séptimo':7, 'octavo': 8, 'noveno': 9, 'decimo': 10, 'décimo': 10, \
                    'decimo primer': 11, 'décimo primer': 11, 'décimoprimer': 11, 'decimoprimer': 11, \
                    'decimo segundo': 12,  'décimo segundo': 12, 'décimosegundo': 12, 'decimosegundo': 12, \
                    'decimo tercer': 13, 'decimotercer': 13,  'décimo tercer': 13, 'décimotercer': 13, \
                    'decimocuarto': 14, 'decimo cuarto': 14, 'décimocuarto': 14, 'décimo cuarto': 14,\
                    'decimoquinto': 15, 'decimo quinto': 15, 'décimoquinto': 15, 'décimo quinto': 15,
                    'décimo sexto': 16,  'decimosexto': 16, 'décimosexto': 16,  'decimo sexto': 16,\
                    'decimoseptimo': 17, 'decimo septimo': 17, 'decimoséptimo': 17, 'decimo séptimo': 17,\
                    'decimooctavo': 18, 'decimo octavo': 18, 'décimooctavo': 18, 'décimo octavo': 18,\
                    'decimonoveno': 19, 'decimo noveno': 19, 'décimonoveno': 19, 'décimo noveno': 19, \
                    'vigesimo': 20, 'vigésimo': 20}
encontradas1 = encontradas.map(num_pisos_mapper)

In [194]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [195]:
data['floor_copia'].isnull().value_counts()

True     90145
False    31075
Name: floor_copia, dtype: int64

In [196]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 20, 30, 40, 50, 1000, 2000]
pisos = pd.cut(data.floor_copia, bins, right = False)
data.pivot_table('floor_copia', index = [pisos], columns= 'property_type_copia', aggfunc={'floor_copia': 'count'})

property_type_copia,PH,apartment,house,store
floor_copia,,,,
"[0, 1)",0,0,0,0
"[1, 2)",879,4324,1431,223
"[2, 3)",659,5035,2182,85
"[3, 4)",149,2417,561,24
"[4, 5)",54,1834,107,21
"[5, 6)",41,1199,36,9
"[6, 7)",45,1336,22,10
"[7, 8)",30,1099,15,7
"[8, 9)",12,1300,13,13


In [197]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [198]:
patron = r"[\s]?piso[\s]?(primer[a|o]?|segundo|tercer|cuarto|quinto|sexto|s[e|é]ptimo|octavo|noveno|d[e|é]cimo|d[e|é]cimo[\s]?primer|d[e|é]cimo[\s]?segundo|d[e|é]cimo[\s]?tercer|d[e|é]cimo[\s]?cuarto|d[e|é]cimo[\s]?quinto|d[e|é]cimo[\s]?sexto|d[e|é]cimo[\s]?s[e|é]ptimo|d[e|é]cimo[\s]?octavo|d[e|é]cimo[\s]?noveno|vig[e|é]simo)"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

num_pisos_mapper = {'primer': 1, 'primera': 1, 'primero': 1, 'segundo':2, 'tercer': 3, 'cuarto': 4, 'quinto': 5, 'sexto': 6, \
                    'septimo':7, 'séptimo':7, 'octavo': 8, 'noveno': 9, 'decimo': 10, 'décimo': 10, \
                    'decimo primer': 11, 'décimo primer': 11, 'décimoprimer': 11, 'decimoprimer': 11, \
                    'decimo segundo': 12,  'décimo segundo': 12, 'décimosegundo': 12, 'decimosegundo': 12, \
                    'decimo tercer': 13, 'decimotercer': 13,  'décimo tercer': 13, 'décimotercer': 13, \
                    'decimocuarto': 14, 'decimo cuarto': 14, 'décimocuarto': 14, 'décimo cuarto': 14,\
                    'decimoquinto': 15, 'decimo quinto': 15, 'décimoquinto': 15, 'décimo quinto': 15,
                    'décimo sexto': 16,  'decimosexto': 16, 'décimosexto': 16,  'decimo sexto': 16,\
                    'decimoseptimo': 17, 'decimo septimo': 17, 'decimoséptimo': 17, 'decimo séptimo': 17,\
                    'decimooctavo': 18, 'decimo octavo': 18, 'décimooctavo': 18, 'décimo octavo': 18,\
                    'decimonoveno': 19, 'decimo noveno': 19, 'décimonoveno': 19, 'décimo noveno': 19, \
                    'vigesimo': 20, 'vigésimo': 20}
encontradas1 = encontradas.map(num_pisos_mapper)

In [199]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [200]:
data['floor_copia'].isnull().value_counts()

True     90118
False    31102
Name: floor_copia, dtype: int64

In [201]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [202]:
patron = r"(un|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez|once|doce|trece|catorce|quince|dieciseis|diecisiete|dieciocho|diecinueve|veinte)[\s]?piso[\s]"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

num_pisos_mapper = {'un': 1,'dos':2, 'tres': 3, 'cuatro': 4, 'cinco': 5, 'seis': 6, \
                    'siete':7, 'ocho': 8, 'nueve': 9, 'diez': 10, \
                    'once': 11, 'doce': 12, 'trece': 13, 'catorce': 14, 'quice': 15, 'dieciseis': 16, \
                    'diecisiete': 17, 'dieciocho': 18, 'diecinueve': 19,\
                    'veinte': 20}
encontradas1 = encontradas.map(num_pisos_mapper)

In [203]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [204]:
data['floor_copia'].isnull().value_counts()

True     89954
False    31266
Name: floor_copia, dtype: int64

In [205]:
bins = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 15, 20, 30, 40, 50, 1000, 2000]
pisos = pd.cut(data.floor_copia, bins, right = False)
data.pivot_table('floor_copia', index = [pisos], columns= 'property_type_copia', aggfunc={'floor_copia': 'count'})

property_type_copia,PH,apartment,house,store
floor_copia,,,,
"[0, 1)",0,0,0,0
"[1, 2)",882,4465,1451,225
"[2, 3)",659,5049,2182,85
"[3, 4)",150,2418,562,24
"[4, 5)",54,1837,107,21
"[5, 6)",41,1201,36,9
"[6, 7)",45,1339,22,10
"[7, 8)",30,1096,15,7
"[8, 9)",12,1302,13,13


In [206]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [207]:
patron = r"(una|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez|once|doce|trece|catorce|quince|dieciseis|diecisiete|dieciocho|diecinueve|veinte)[\s]?[sola]?[\s]?planta[s]?"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

num_pisos_mapper = {'una': 0,'dos':2, 'tres': 3, 'cuatro': 4, 'cinco': 5, 'seis': 6, \
                    'siete':7, 'ocho': 8, 'nueve': 9, 'diez': 10, \
                    'once': 11, 'doce': 12, 'trece': 13, 'catorce': 14, 'quice': 15, 'dieciseis': 16, \
                    'diecisiete': 17, 'dieciocho': 18, 'diecinueve': 19,\
                    'veinte': 20}
encontradas1 = encontradas.map(num_pisos_mapper)

In [208]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [209]:
data['floor_copia'].isnull().value_counts()

True     82318
False    38902
Name: floor_copia, dtype: int64

In [210]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [211]:
patron = r"planta[\s]?(un[a|o]|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez|once|doce|trece|catorce|quince|dieciseis|diecisiete|dieciocho|diecinueve|veinte)"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

num_pisos_mapper = {'una': 1, 'uno': 1, 'dos':2, 'tres': 3, 'cuatro': 4, 'cinco': 5, 'seis': 6, \
                    'siete':7, 'ocho': 8, 'nueve': 9, 'diez': 10, \
                    'once': 11, 'doce': 12, 'trece': 13, 'catorce': 14, 'quice': 15, 'dieciseis': 16, \
                    'diecisiete': 17, 'dieciocho': 18, 'diecinueve': 19,\
                    'veinte': 20}
encontradas1 = encontradas.map(num_pisos_mapper)

In [212]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [213]:
data['floor_copia'].isnull().value_counts()

True     82280
False    38940
Name: floor_copia, dtype: int64

In [214]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [215]:
patron = r"(planta[\s]?alta)"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

numcol_cat_mapper = {'planta alta': 1,'plantalata': 1}
encontradas1 = encontradas.map(numcol_cat_mapper)

In [216]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [217]:
data['floor_copia'].isnull().value_counts()

True     74322
False    46898
Name: floor_copia, dtype: int64

In [218]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [219]:
patron = r"(planta[\s]?baja)"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

numcol_cat_mapper = {'planta baja': 0,'plantabaja': 0}
encontradas1 = encontradas.map(numcol_cat_mapper)

In [220]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [221]:
data['floor_copia'].isnull().value_counts()

True     68408
False    52812
Name: floor_copia, dtype: int64

In [222]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [223]:
mascara = data['description_copia'] == ""

Ahora busco con las mismas regex pero con la columna 'title_copia'

In [224]:
patron = r"(planta[\s]?baja)"
busca = re.compile(patron)

description_series_match = data.title_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

numcol_cat_mapper = {'planta baja': 0,'plantabaja': 0}
encontradas1 = encontradas.map(numcol_cat_mapper)

In [225]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [226]:
data['floor_copia'].isnull().value_counts()

True     68306
False    52914
Name: floor_copia, dtype: int64

In [227]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [228]:
patron = r"(una|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez|once|doce|trece|catorce|quince|dieciseis|diecisiete|dieciocho|diecinueve|veinte)[\s]?[sola]?[\s]?planta[s]?"
busca = re.compile(patron)

description_series_match = data.title_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

num_pisos_mapper = {'una': 0,'dos':2, 'tres': 3, 'cuatro': 4, 'cinco': 5, 'seis': 6, \
                    'siete':7, 'ocho': 8, 'nueve': 9, 'diez': 10, \
                    'once': 11, 'doce': 12, 'trece': 13, 'catorce': 14, 'quice': 15, 'dieciseis': 16, \
                    'diecisiete': 17, 'dieciocho': 18, 'diecinueve': 19,\
                    'veinte': 20}
encontradas1 = encontradas.map(num_pisos_mapper)

In [229]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [230]:
data['floor_copia'].isnull().value_counts()

True     68229
False    52991
Name: floor_copia, dtype: int64

In [231]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [232]:
patron = r"(un|dos|tres|cuatro|cinco|seis|siete|ocho|nueve|diez|once|doce|trece|catorce|quince|dieciseis|diecisiete|dieciocho|diecinueve|veinte)[\s]?piso[\s]"
busca = re.compile(patron)

description_series_match = data.title_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

num_pisos_mapper = {'un': 1,'dos':2, 'tres': 3, 'cuatro': 4, 'cinco': 5, 'seis': 6, \
                    'siete':7, 'ocho': 8, 'nueve': 9, 'diez': 10, \
                    'once': 11, 'doce': 12, 'trece': 13, 'catorce': 14, 'quice': 15, 'dieciseis': 16, \
                    'diecisiete': 17, 'dieciocho': 18, 'diecinueve': 19,\
                    'veinte': 20}
encontradas1 = encontradas.map(num_pisos_mapper)

In [233]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [234]:
data['floor_copia'].isnull().value_counts()

True     68227
False    52993
Name: floor_copia, dtype: int64

In [235]:
patron = r"(primer|segundo|tercer|cuarto|quinto|sexto|s[e|é]ptimo|octavo|noveno|d[e|é]cimo|d[e|é]cimo[\s]?primer|d[e|é]cimo[\s]?segundo|d[e|é]cimo[\s]?tercer|d[e|é]cimo[\s]?cuarto|d[e|é]cimo[\s]?quinto|d[e|é]cimo[\s]?sexto|d[e|é]cimo[\s]?s[e|é]ptimo|d[e|é]cimo[\s]?octavo|d[e|é]cimo[\s]?noveno|vig[e|é]simo)[\s]?piso"
busca = re.compile(patron)

description_series_match = data.title_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

num_pisos_mapper = {'primer': 1,'segundo':2, 'tercer': 3, 'cuarto': 4, 'quinto': 5, 'sexto': 6, \
                    'septimo':7, 'séptimo':7, 'octavo': 8, 'noveno': 9, 'decimo': 10, 'décimo': 10, \
                    'decimo primer': 11, 'décimo primer': 11, 'décimoprimer': 11, 'decimoprimer': 11, \
                    'decimo segundo': 12,  'décimo segundo': 12, 'décimosegundo': 12, 'decimosegundo': 12, \
                    'decimo tercer': 13, 'decimotercer': 13,  'décimo tercer': 13, 'décimotercer': 13, \
                    'decimocuarto': 14, 'decimo cuarto': 14, 'décimocuarto': 14, 'décimo cuarto': 14,\
                    'decimoquinto': 15, 'decimo quinto': 15, 'décimoquinto': 15, 'décimo quinto': 15,
                    'décimo sexto': 16,  'decimosexto': 16, 'décimosexto': 16,  'decimo sexto': 16,\
                    'decimoseptimo': 17, 'decimo septimo': 17, 'decimoséptimo': 17, 'decimo séptimo': 17,\
                    'decimooctavo': 18, 'decimo octavo': 18, 'décimooctavo': 18, 'décimo octavo': 18,\
                    'decimonoveno': 19, 'decimo noveno': 19, 'décimonoveno': 19, 'décimo noveno': 19, \
                    'vigesimo': 20, 'vigésimo': 20}
encontradas1 = encontradas.map(num_pisos_mapper)

In [236]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [237]:
data['floor_copia'].isnull().value_counts()

True     68187
False    53033
Name: floor_copia, dtype: int64

In [238]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [239]:
patron = r"(duplex|triplex)"
busca = re.compile(patron)

description_series_match = data.title_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

numcol_cat_mapper = {'duplex': 2,'triplex': 3}
encontradas1 = encontradas.map(numcol_cat_mapper)

In [240]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [241]:
data['floor_copia'].isnull().value_counts()

True     67932
False    53288
Name: floor_copia, dtype: int64

In [242]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [243]:
patron = "\s(?P<numero>\d[\d]?)\s*\w{0,5}\s*piso"
busca = re.compile(patron)

description_series_match = data.title_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))

mask_description_notnull = description_series_match.notnull()
data.loc[mask_description_notnull,'floor_copia'] = description_series_match[mask_description_notnull].apply(lambda x: x.group('numero')).astype(float)

In [244]:
data['floor_copia'].isnull().value_counts()

True     67789
False    53431
Name: floor_copia, dtype: int64

In [245]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [246]:
patron = "\s(?P<numero>\d[\d]?)[\s]?[º|°]?\s*piso"
busca = re.compile(patron)

description_series_match = data.title_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))

mask_description_notnull = description_series_match.notnull()
data.loc[mask_description_notnull,'floor_copia'] = description_series_match[mask_description_notnull].apply(lambda x: x.group('numero')).astype(float)

In [247]:
data['floor_copia'].isnull().value_counts()

True     67747
False    53473
Name: floor_copia, dtype: int64

In [248]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [249]:
patron = "piso[\s]?(?P<numero>\d[\d]?)"

busca = re.compile(patron)

description_series_match = data.title_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))

mask_description_notnull = description_series_match.notnull()
data.loc[mask_description_notnull,'floor_copia'] = description_series_match[mask_description_notnull].apply(lambda x: x.group('numero')).astype(float)

In [250]:
data['floor_copia'].isnull().value_counts()

True     67478
False    53742
Name: floor_copia, dtype: int64

In [251]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [252]:
patron = "\spiso:[\s]?(?P<numero>\d[\d]?)"
busca = re.compile(patron)

description_series_match = data.title_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))

mask_description_notnull = description_series_match.notnull()
data.loc[mask_description_notnull,'floor_copia'] = description_series_match[mask_description_notnull].apply(lambda x: x.group('numero')).astype(float)

In [253]:
data['floor_copia'].isnull().value_counts()

True     67478
False    53742
Name: floor_copia, dtype: int64

In [254]:
data.loc[data.title_copia!= ""]

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,rooms,expenses,properati_url,description,title,image_thumbnail,floor_copia,property_type_copia,description_copia,title_copia
4,4,sell,apartment,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-38.0026256,-57.5494468",-38.002626,...,NaN,NaN,http://www.properati.com.ar/15bok_venta_depart...,DEPARTAMENTO CON FANTÁSTICA ILUMINACIÓN NATURA...,DEPTO 2 AMB AL CONTRAFRENTE ZONA CENTRO/PLAZA ...,https://thumbs4.properati.com/5/xrRqlNcSI_vs-f...,NaN,apartment,departamento con fantástica iluminación natura...,depto 2 amb al contrafrente zona centro/plaza ...
5,5,sell,house,Gualeguaychú,|Argentina|Entre Ríos|Gualeguaychú|,Argentina,Entre Ríos,3433657.0,"-33.0140714,-58.519828",-33.014071,...,NaN,NaN,http://www.properati.com.ar/15bop_venta_depart...,"Casa en el perímetro del barrio 338, ubicada e...","Casa Barrio 338. Sobre calle 3 de caballería, ...",https://thumbs4.properati.com/6/q-w68gvaUEQVXI...,NaN,house,"casa en el perímetro del barrio 338, ubicada e...","casa barrio 338. sobre calle 3 de caballería, ..."
7,7,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,...,NaN,NaN,http://www.properati.com.ar/15bot_venta_depart...,EXCELENTE MONOAMBIENTE A ESTRENAR AMPLIO SUPER...,JOSE HERNANDEZ 1400 MONOAMBIENTE ESTRENAR CAT...,https://thumbs4.properati.com/1/IHxARynlr8sPEW...,NaN,apartment,excelente monoambiente a estrenar amplio super...,jose hernandez 1400 monoambiente estrenar cat...
8,8,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,"-34.5598729,-58.443362",-34.559873,...,NaN,NaN,http://www.properati.com.ar/15bou_venta_depart...,EXCELENTE DOS AMBIENTES ESTRENAR AMPLIO SUPER...,"JOSE HERNANDEZ 1400 DOS AMBIENTES ESTRENAR ,...",https://thumbs4.properati.com/2/J3zOjgaFHrkvnv...,NaN,apartment,excelente dos ambientes estrenar amplio super...,"jose hernandez 1400 dos ambientes estrenar ,..."
10,10,sell,house,Córdoba,|Argentina|Córdoba|Córdoba|,Argentina,Córdoba,3860259.0,"-31.4200833,-64.1887761",-31.420083,...,NaN,NaN,http://www.properati.com.ar/15boy_venta_casa_c...,"Hermosa casa sin terminar de 3 dormitorios, ba...",Casa sin terminar en Falda de Cañete. ...,https://thumbs4.properati.com/3/mQkhIxXpTTbMSc...,NaN,house,"hermosa casa sin terminar de 3 dormitorios, ba...",casa sin terminar en falda de cañete. ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121214,121214,sell,store,San Isidro,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro|,Argentina,Bs.As. G.B.A. Zona Norte,3428983.0,NaN,NaN,...,NaN,NaN,http://www.properati.com.ar/1cja1_venta_local_...,***VENTA CON RENTA***Local en EDIFICIO lomas ...,Local - San Isidro,https://thumbs4.properati.com/7/R-j0JiSePtAriH...,NaN,store,***venta con renta***local en edificio lomas ...,local - san isidro
121215,121215,sell,apartment,Belgrano,|Argentina|Capital Federal|Belgrano|,Argentina,Capital Federal,3436077.0,NaN,NaN,...,NaN,10000.0,http://www.properati.com.ar/1cja2_venta_depart...,TORRE FORUM ALCORTA - MÁXIMA CATEGORÍA.Impecab...,Torre Forum Alcorta- Impecable 3 ambientes,https://thumbs4.properati.com/1/bjms0KnaAnlNoQ...,NaN,apartment,torre forum alcorta - máxima categoría.impecab...,torre forum alcorta- impecable 3 ambientes
121216,121216,sell,house,Beccar,|Argentina|Bs.As. G.B.A. Zona Norte|San Isidro...,Argentina,Bs.As. G.B.A. Zona Norte,3436080.0,NaN,NaN,...,NaN,NaN,http://www.properati.com.ar/1cja6_venta_casa_b...,Excelente e impecable casa en Venta en Las Lom...,Ruca Inmuebles | Venta | Lomas de San Isidro |...,https://thumbs4.properati.com/2/PCc3WuQDjpNZc4...,NaN,house,excelente e impecable casa en venta en las lom...,ruca inmuebles | venta | lomas de san isidro |...
121217,121217,sell,apartment,Villa Urquiza,|Argentina|Capital Federal|Villa Ur

In [255]:
data.loc[121217, 'description_copia']

'venta departamento ambiente divisible a estrenar balcon amenities.entrega inmediata* departamento: excelente monoambiente de (43,00m2) divisible, con importante placard - baño completo - balcón corrido, cocina integrada con equipamiento completo y herrajes de acero inoxidable, grifería fv, cocina con horno, termotanque eléctrico, servicios individuales, preinstalación aire acondicionado frío-calor, conexión lavarropas y lavavajillas.* edificio: moderno edificio, amenities - laundry - parrillas y solarium - terraza de uso común - grupo electrógeno.* cocheras disponibles desde usd 20.000* barrio: nuevo corredor urbano de 13 manzanas - holmberg/donado - área parquizada  - bicisendas  - zona de importantes embajadas - rodeado de espacios verdes.* medios de transporte: siete cuadras del subte b - próximo a uba( drago), colegio san patrick, estaciones drago y coglhan.si tenes que vender para comprar // somos especialistas en operaciones simultaneas// comunicate con nosotros para asesorarte.

In [256]:
patron = r"(casa[s]?|chalet|quinta[s]?|cabaña[s]?|pb)"
busca = re.compile(patron)

description_series_match = data.title_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

num_pisos_mapper = {'casa': 0,'casas':0, 'chalet': 0, 'quinta': 0, 'quintas': 0, 'cabaña': 0, \
                    'cabañas':0, 'pb': 0 }
encontradas1 = encontradas.map(num_pisos_mapper)

In [257]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [258]:
data['floor_copia'].isnull().value_counts()

False    72785
True     48435
Name: floor_copia, dtype: int64

In [259]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""

In [260]:
patron = r"(casa[s]?|chalet|quinta[s]?|cabaña[s]?|pb)"
busca = re.compile(patron)

description_series_match = data.description_copia.apply(lambda x:  x if x is np.NaN else busca.search(x))
mask_description_notnull = description_series_match.notnull()
encontradas = description_series_match[mask_description_notnull].apply(lambda x: x.group(1))

num_pisos_mapper = {'casa': 0,'casas':0, 'chalet': 0, 'quinta': 0, 'quintas': 0, 'cabaña': 0, \
                    'cabañas':0, 'pb': 0 }
encontradas1 = encontradas.map(num_pisos_mapper)

In [261]:
data.loc[mask_description_notnull,'floor_copia']= encontradas1.astype(float)

In [262]:
data['floor_copia'].isnull().value_counts()

False    77585
True     43635
Name: floor_copia, dtype: int64

In [263]:
data.loc[mask_description_notnull, ['description_copia', 'title_copia']] = ""